# 04 - Company Name Standardization

## Objective
Standardize company names to consolidate variations and improve data quality for analysis.

## Tasks
- Identify company name variations (case, suffixes, subsidiaries)
- Create comprehensive company mapping dictionary
- Apply standardization rules systematically
- Verify consolidation results
- Handle edge cases (typos, abbreviations, subsidiaries)

## Expected Output
- Standardized company names in new column: `company_clean`
- Reduced unique company count (1,908 → ~1,700-1,800)
- Mapping documentation for transparency
- Updated dataset: `data/processed/jobs_cleaned_with_std_companies.csv`


## 1. Environment Setup


In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter, defaultdict
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print(" Libraries imported successfully")
print(f"   Pandas version: {pd.__version__}")


## 2. Load Cleaned Data


In [ ]:
print(" Loading cleaned dataset...\n")

df = pd.read_csv('data/processed/jobs_cleaned.csv')

print("="*70)
print(" DATASET LOADED")
print("="*70)

print(f"\n Dataset Overview:")
print(f"   Total records: {len(df):,}")
print(f"   Unique companies: {df['company'].nunique():,}")
print(f"   Date range: {df['posting_date'].min()} to {df['posting_date'].max()}")

print(f"\n🏢 Top 15 Companies (current):")
top_15 = df['company'].value_counts().head(15)
for idx, (company, count) in enumerate(top_15.items(), 1):
    print(f"   {idx:2d}. {company[:45]:<45} {count:>4} jobs")


## 3. Analyze Company Name Patterns


In [ ]:
def analyze_company_patterns(df):
    """Analyze company name patterns and identify variations"""
    
    print("="*70)
    print(" COMPANY NAME PATTERN ANALYSIS")
    print("="*70)
    
    # Case sensitivity analysis
    print("\n1️⃣  Case Sensitivity Issues:")
    company_normalized = {}
    for company in df['company'].unique():
        lower = company.lower()
        if lower not in company_normalized:
            company_normalized[lower] = []
        company_normalized[lower].append(company)
    
    case_issues = {k: v for k, v in company_normalized.items() if len(v) > 1}
    print(f"   Found {len(case_issues)} companies with case variations")
    
    case_mappings = {}
    for lower_name, variations in case_issues.items():
        counts = [(v, (df['company'] == v).sum()) for v in variations]
        primary = max(counts, key=lambda x: x[1])[0]
        for var, _ in counts:
            if var != primary:
                case_mappings[var] = primary
    
    print(f"   Will consolidate {len(case_mappings)} variations")
    
    # Suffix analysis
    print("\n2️⃣  Legal Suffix Variations:")
    suffix_patterns = [
        r',?\s+(LLC|Inc\.?|Corp\.?|Corporation|Ltd\.?|Limited|LLP|LP)$',
        r',?\s+(Co\.?|Company|PLC|GmbH|SA|AG)$'
    ]
    
    companies_with_suffixes = 0
    for company in df['company'].unique():
        for pattern in suffix_patterns:
            if re.search(pattern, company, re.IGNORECASE):
                companies_with_suffixes += 1
                break
    
    print(f"   Companies with legal suffixes: {companies_with_suffixes}")
    
    # Major tech company variations
    print("\n3️⃣  Major Tech Company Variations:")
    tech_checks = {
        'Amazon': ['amazon'],
        'Microsoft': ['microsoft'],
        'Google': ['google', 'alphabet'],
        'Meta': ['meta', 'facebook'],
        'Apple': ['apple'],
        'Oracle': ['oracle'],
        'IBM': ['ibm'],
        'Adobe': ['adobe'],
        'NVIDIA': ['nvidia'],
    }
    
    tech_variations = {}
    for tech_name, keywords in tech_checks.items():
        matches = []
        for company in df['company'].unique():
            company_lower = company.lower()
            # Exact or starts with tech name
            if any(company_lower == kw or company_lower.startswith(kw + ' ') for kw in keywords):
                count = (df['company'] == company).sum()
                matches.append((company, count))
        
        if len(matches) > 1:
            tech_variations[tech_name] = matches
            total = sum(c for _, c in matches)
            print(f"   {tech_name}: {total} jobs across {len(matches)} variations")
            for company, count in sorted(matches, key=lambda x: -x[1])[:3]:
                print(f"      → {company[:50]:<50} {count:>3} jobs")
    
    return case_mappings, tech_variations

case_mappings, tech_variations = analyze_company_patterns(df)


## 4. Build Standardization Functions


In [ ]:
def remove_legal_suffixes(company_name):
    """Remove common legal suffixes from company names"""
    if pd.isna(company_name):
        return company_name
    
    suffixes = [
        r',?\s+LLC$',
        r',?\s+Inc\.?$',
        r',?\s+Corp\.?$',
        r',?\s+Corporation$',
        r',?\s+Ltd\.?$',
        r',?\s+Limited$',
        r',?\s+LLP$',
        r',?\s+LP$',
        r',?\s+Co\.?$',
        r',?\s+Company$',
        r',?\s+PLC$',
        r',?\s+GmbH$',
        r',?\s+SA$',
        r',?\s+AG$',
    ]
    
    cleaned = company_name
    for pattern in suffixes:
        cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
    
    cleaned = cleaned.strip().strip(',').strip()
    
    return cleaned if cleaned else company_name


def standardize_spacing(company_name):
    """Normalize spacing in company names"""
    if pd.isna(company_name):
        return company_name
    
    cleaned = re.sub(r'\s+', ' ', company_name)
    
    cleaned = cleaned.strip()
    
    return cleaned


def apply_title_case(company_name):
    """Apply intelligent title casing"""
    if pd.isna(company_name):
        return company_name
    
    keep_upper = ['IBM', 'JPMorgan', 'CVS', 'BMW', 'AWS', 'NASA', 
                  'FBI', 'CIA', 'NFL', 'NBA', 'NVIDIA', 'AMD']
    
    for term in keep_upper:
        if company_name.upper() == term.upper():
            return term
    
    exceptions = ['and', 'of', 'the', 'for', 'in', 'on', 'at', 'to', 'a', 'an']
    
    words = company_name.split()
    result = []
    
    for i, word in enumerate(words):
        if word.upper() in keep_upper:
            result.append(word.upper())
        elif i > 0 and word.lower() in exceptions:
            result.append(word.lower())
        else:
            result.append(word.capitalize())
    
    return ' '.join(result)


print(" Standardization functions defined:")
print("   • remove_legal_suffixes()")
print("   • standardize_spacing()")
print("   • apply_title_case()")


## 5. Create Company Mapping Dictionary


In [ ]:
print("🗺️  Building comprehensive company mapping dictionary...\n")

company_mapping = {}

# 1. Tech Giants and Subsidiaries
tech_mapping = {
    # Amazon
    'Amazon Stores': 'Amazon',
    'Amazon Development Center U.S., Inc.': 'Amazon',
    'Amazon Web Services, Inc.': 'Amazon',
    'Amazon Web Services': 'Amazon',
    'AWS': 'Amazon',
    
    # Microsoft
    'Microsoft Corporation': 'Microsoft',
    'MSFT': 'Microsoft',
    
    # Google/Alphabet
    'Alphabet Inc.': 'Google',
    'Alphabet': 'Google',
    'Google LLC': 'Google',
    
    # Meta/Facebook
    'Facebook': 'Meta',
    'Meta Platforms': 'Meta',
    'Meta Platforms, Inc.': 'Meta',
    'Meta Reality Labs': 'Meta',
    'Meta Reality Labs - Conversational AI': 'Meta',
    
    # Apple
    'Apple Inc.': 'Apple',
    'Apple Computer': 'Apple',
    
    # Oracle
    'Oracle Corporation': 'Oracle',
    
    # Adobe
    'Adobe Inc.': 'Adobe',
    'Adobe Systems': 'Adobe',
    
    # Salesforce
    'Salesforce.com': 'Salesforce',
    
    # IBM
    'International Business Machines': 'IBM',
    
    # Intel
    'Intel Corporation': 'Intel',
    
    # NVIDIA
    'NVIDIA Corporation': 'NVIDIA',
    'Nvidia': 'NVIDIA',
    
    # Tesla
    'Tesla Motors': 'Tesla',
    'Tesla Inc.': 'Tesla',
    
    # Netflix
    'Netflix Inc.': 'Netflix',
    
    # Uber
    'Uber Technologies': 'Uber',
    'Uber Technologies, Inc.': 'Uber',
}

company_mapping.update(tech_mapping)

# 2. Consulting Firms
consulting_mapping = {
    'Deloitte Consulting': 'Deloitte',
    'Deloitte LLP': 'Deloitte',
    'DELOITTE': 'Deloitte',
    'Deloitte & Touche': 'Deloitte',
    
    'Accenture Federal Services': 'Accenture',
    'Accenture LLP': 'Accenture',
    
    'Boston Consulting Group (BCG)': 'Boston Consulting Group',
    'BCG': 'Boston Consulting Group',
    
    'McKinsey & Company': 'McKinsey',
    'McKinsey': 'McKinsey',
    
    'PwC': 'PricewaterhouseCoopers',
    'PricewaterhouseCoopers LLP': 'PricewaterhouseCoopers',
    
    'EY': 'Ernst & Young',
    'Ernst & Young LLP': 'Ernst & Young',
    
    'KPMG LLP': 'KPMG',
}

company_mapping.update(consulting_mapping)

# 3. Aerospace & Defense
aerospace_mapping = {
    'Boeing Company': 'Boeing',
    'BOEING': 'Boeing',
    'The Boeing Company': 'Boeing',
    
    'Lockheed Martin Corporation': 'Lockheed Martin',
    'Lockheed Martin Corp.': 'Lockheed Martin',
    
    'Raytheon Technologies': 'RTX',
    'RTX Corporation': 'RTX',
    'Raytheon': 'RTX',
    
    'Northrop Grumman Corporation': 'Northrop Grumman',
    
    'General Dynamics Corporation': 'General Dynamics',
}

company_mapping.update(aerospace_mapping)

# 4. Healthcare
healthcare_mapping = {
    'CVS HEALTH': 'CVS Health',
    'CVS Pharmacy': 'CVS Health',
    'CVS': 'CVS Health',
    
    'UnitedHealth Group': 'UnitedHealth',
    'United Healthcare': 'UnitedHealth',
    
    'Kaiser Permanente': 'Kaiser',
    'Kaiser Foundation': 'Kaiser',
}

company_mapping.update(healthcare_mapping)

# 5. Financial Services
finance_mapping = {
    'JPMorganChase': 'JPMorgan Chase',
    'JPMorgan Chase & Co.': 'JPMorgan Chase',
    'JP Morgan': 'JPMorgan Chase',
    'Chase': 'JPMorgan Chase',
    
    'Bank of America Corporation': 'Bank of America',
    'BofA': 'Bank of America',
    
    'Goldman Sachs Group': 'Goldman Sachs',
    'Goldman Sachs & Co.': 'Goldman Sachs',
    
    'Morgan Stanley & Co.': 'Morgan Stanley',
    
    'Wells Fargo & Company': 'Wells Fargo',
    'Wells Fargo Bank': 'Wells Fargo',
}

company_mapping.update(finance_mapping)

# 6. Automotive
auto_mapping = {
    'General Motors Company': 'General Motors',
    'GM': 'General Motors',
    
    'Ford Motor Company': 'Ford',
    'Ford Motor': 'Ford',
}

company_mapping.update(auto_mapping)

# 7. Telecommunications
telecom_mapping = {
    'AT&T Inc.': 'AT&T',
    'ATT': 'AT&T',
    
    'Verizon Communications': 'Verizon',
    'Verizon Wireless': 'Verizon',
    
    'T-Mobile USA': 'T-Mobile',
    'TMobile': 'T-Mobile',
}

company_mapping.update(telecom_mapping)

# 8. Retail
retail_mapping = {
    'Walmart Inc.': 'Walmart',
    'Wal-Mart': 'Walmart',
    
    'Target Corporation': 'Target',
}

company_mapping.update(retail_mapping)

# Add case-based mappings (from analysis)
company_mapping.update(case_mappings)

print(f" Company mapping dictionary created")
print(f"   Total mappings: {len(company_mapping)}")
print(f"\n   Categories:")
print(f"      • Tech giants & subsidiaries: {len(tech_mapping)}")
print(f"      • Consulting firms: {len(consulting_mapping)}")
print(f"      • Aerospace & defense: {len(aerospace_mapping)}")
print(f"      • Healthcare: {len(healthcare_mapping)}")
print(f"      • Financial services: {len(finance_mapping)}")
print(f"      • Automotive: {len(auto_mapping)}")
print(f"      • Telecommunications: {len(telecom_mapping)}")
print(f"      • Retail: {len(retail_mapping)}")
print(f"      • Case sensitivity fixes: {len(case_mappings)}")


In [ ]:
def standardize_company_name(company_name):
    """
    Apply full standardization pipeline to a company name
    
    Steps:
    1. Apply explicit mapping (if exists)
    2. Standardize spacing
    3. Remove legal suffixes
    4. Apply title casing
    """
    if pd.isna(company_name):
        return company_name
    
    # Step 1: Check if we have an explicit mapping
    if company_name in company_mapping:
        return company_mapping[company_name]
    
    # Step 2: Standardize spacing
    cleaned = standardize_spacing(company_name)
    
    # Step 3: Remove legal suffixes
    cleaned = remove_legal_suffixes(cleaned)
    
    # Step 4: Apply title casing
    cleaned = apply_title_case(cleaned)
    
    # Check mapping again after cleaning
    if cleaned in company_mapping:
        return company_mapping[cleaned]
    
    return cleaned


print("🔄 Applying standardization pipeline...\n")

# Create the cleaned company column
df['company_clean'] = df['company'].apply(standardize_company_name)

print(" Standardization complete!")
print(f"\n   Original unique companies: {df['company'].nunique():,}")
print(f"   Standardized unique companies: {df['company_clean'].nunique():,}")
print(f"   Reduction: {df['company'].nunique() - df['company_clean'].nunique():,} companies consolidated")


## 7. Verification & Quality Checks


In [ ]:
print("="*70)
print(" STANDARDIZATION VERIFICATION")
print("="*70)

# Check 1: Compare top companies before/after
print("\n1️⃣  Top 15 Companies Comparison:")
print("\n   BEFORE standardization:")
top_before = df['company'].value_counts().head(15)
for idx, (company, count) in enumerate(top_before.items(), 1):
    print(f"   {idx:2d}. {company[:45]:<45} {count:>4} jobs")

print("\n   AFTER standardization:")
top_after = df['company_clean'].value_counts().head(15)
for idx, (company, count) in enumerate(top_after.items(), 1):
    print(f"   {idx:2d}. {company[:45]:<45} {count:>4} jobs")

# Check 2: Show specific consolidations
print("\n\n2️⃣  Specific Consolidation Examples:")

examples = [
    'Boeing',
    'Deloitte', 
    'Microsoft',
    'Amazon',
    'Meta',
    'CVS Health',
]

for company in examples:
    before_variations = df[df['company_clean'] == company]['company'].unique()
    after_count = (df['company_clean'] == company).sum()
    
    if len(before_variations) > 1:
        print(f"\n   {company}:")
        print(f"      Total jobs: {after_count}")
        print(f"      Consolidated from {len(before_variations)} variations:")
        for var in sorted(before_variations):
            count = (df['company'] == var).sum()
            print(f"         • {var[:50]:<50} ({count} jobs)")

# Check 3: Verify no data loss
print("\n\n3️⃣  Data Integrity Check:")
null_before = df['company'].isnull().sum()
null_after = df['company_clean'].isnull().sum()
print(f"   Null values before: {null_before}")
print(f"   Null values after: {null_after}")
print(f"    No data loss" if null_before == null_after else "   ⚠️  Data loss detected!")

# Check 4: Suffix removal effectiveness
print("\n4️⃣  Suffix Removal Effectiveness:")
suffixes_before = df['company'].str.contains(r'(LLC|Inc\.?|Corp\.?|Corporation|Ltd\.?)$', 
                                              case=False, na=False, regex=True).sum()
suffixes_after = df['company_clean'].str.contains(r'(LLC|Inc\.?|Corp\.?|Corporation|Ltd\.?)$',
                                                   case=False, na=False, regex=True).sum()
print(f"   Companies with suffixes before: {suffixes_before}")
print(f"   Companies with suffixes after: {suffixes_after}")
print(f"   Suffixes removed: {suffixes_before - suffixes_after}")


## 8. Edge Case Analysis


In [ ]:
print("="*70)
print(" EDGE CASE ANALYSIS")
print("="*70)

# Find potential issues or missed consolidations
print("\n1️⃣  Companies with Very Similar Names (Potential Missed Matches):")

from difflib import SequenceMatcher

def find_similar_companies(companies, threshold=0.85):
    """Find pairs of companies with similar names"""
    similar_pairs = []
    company_list = companies.tolist()
    
    for i in range(len(company_list)):
        for j in range(i + 1, len(company_list)):
            if SequenceMatcher(None, company_list[i].lower(), 
                             company_list[j].lower()).ratio() > threshold:
                count_i = (companies == company_list[i]).sum()
                count_j = (companies == company_list[j]).sum()
                if count_i > 1 or count_j > 1:  # Only show if significant
                    similar_pairs.append((company_list[i], company_list[j], count_i, count_j))
    
    return similar_pairs

print("   Checking for similar company names (>85% similarity)...")
similar = find_similar_companies(df['company_clean'].value_counts().head(100).index, threshold=0.85)

if similar:
    print(f"   Found {len(similar)} potential matches to review:\n")
    for comp1, comp2, count1, count2 in similar[:10]:
        print(f"      • {comp1[:40]:<40} ({count1} jobs)")
        print(f"        vs {comp2[:40]:<40} ({count2} jobs)")
        print()
else:
    print("    No obvious similar names detected")

# Check for extremely short names (might be abbreviations)
print("\n2️⃣  Very Short Company Names (Potential Abbreviations):")
short_names = df[df['company_clean'].str.len() <= 4]['company_clean'].value_counts().head(10)
if len(short_names) > 0:
    print("   Companies with ≤4 characters:")
    for company, count in short_names.items():
        print(f"      • {company:<10} ({count} jobs)")
else:
    print("    No concerning short names found")

# Check for "Not specified"
print("\n3️⃣  Missing/Invalid Company Names:")
not_specified = (df['company_clean'] == 'Not specified').sum()
print(f"   'Not specified': {not_specified} jobs")
if not_specified > 0:
    print(f"   ({not_specified/len(df)*100:.2f}% of dataset)")


## 9. Generate Mapping Documentation


In [ ]:
print(" Generating mapping documentation...\n")

# Create a comprehensive mapping log
mapping_log = []

for original, cleaned in zip(df['company'], df['company_clean']):
    if original != cleaned:
        mapping_log.append({
            'original': original,
            'standardized': cleaned,
            'transformation': 'explicit_mapping' if original in company_mapping else 'automated_cleaning'
        })

mapping_df = pd.DataFrame(mapping_log)

if len(mapping_df) > 0:
    mapping_df = mapping_df.drop_duplicates()
    mapping_summary = mapping_df.groupby('standardized')['original'].apply(list).reset_index()
    mapping_summary.columns = ['Standardized_Name', 'Original_Variations']
    mapping_summary['Variation_Count'] = mapping_summary['Original_Variations'].apply(len)
    mapping_summary = mapping_summary.sort_values('Variation_Count', ascending=False)
    
    # Save mapping documentation
    mapping_summary.to_csv('data/processed/company_name_mappings.csv', index=False)
    
    print(f" Mapping documentation saved")
    print(f"   File: data/processed/company_name_mappings.csv")
    print(f"   Total mappings: {len(mapping_df)}")
    print(f"   Unique transformations: {len(mapping_summary)}")
    
    print(f"\n Top 10 Companies with Most Variations:")
    for idx, row in mapping_summary.head(10).iterrows():
        print(f"\n   {row['Standardized_Name']} ({row['Variation_Count']} variations):")
        for var in row['Original_Variations'][:5]:
            print(f"      • {var}")
        if row['Variation_Count'] > 5:
            print(f"      ... and {row['Variation_Count'] - 5} more")
else:
    print("   ℹ️  No mappings to document (all names were already standardized)")


## 10. Save Standardized Dataset


In [ ]:
import os

output_file = 'data/processed/jobs_with_standardized_companies.csv'

df.to_csv(output_file, index=False)

print(" STANDARDIZED DATASET SAVED")
print("="*70)
print(f"\n   File: {output_file}")
print(f"   Size: {os.path.getsize(output_file) / 1024 / 1024:.2f} MB")
print(f"   Records: {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print(f"\n   New column added: 'company_clean'")

print(f"\n Phase 2.2 Complete: Company Name Standardization")
print(f"\n   Dataset ready for exploratory data analysis")


## 11. Final Summary & Statistics


In [ ]:
print("\n" + "="*70)
print(" COMPANY STANDARDIZATION SUMMARY")
print("="*70)

print(f"\n1️  Overall Impact:")
print(f"   Original unique companies: {df['company'].nunique():,}")
print(f"   Standardized unique companies: {df['company_clean'].nunique():,}")
reduction = df['company'].nunique() - df['company_clean'].nunique()
reduction_pct = (reduction / df['company'].nunique()) * 100
print(f"   Companies consolidated: {reduction:,} ({reduction_pct:.1f}%)")

print(f"\n2️  Standardization Methods Applied:")
print(f"   • Explicit mappings: {len(company_mapping)} rules")
print(f"   • Legal suffix removal")
print(f"   • Case standardization")
print(f"   • Spacing normalization")

print(f"\n3️ Top 20 Companies (After Standardization):")
top_20_final = df['company_clean'].value_counts().head(20)
for idx, (company, count) in enumerate(top_20_final.items(), 1):
    pct = (count / len(df)) * 100
    print(f"   {idx:2d}. {company[:40]:<40} {count:>4} jobs ({pct:>5.2f}%)")

print(f"\n4️  Data Quality Metrics:")
print(f"   Total records: {len(df):,}")
print(f"   Records with valid companies: {(df['company_clean'] != 'Not specified').sum():,}")
print(f"   Data completeness: {(df['company_clean'] != 'Not specified').sum()/len(df)*100:.1f}%")

print(f"\n5️  Industry Representation (Top Companies):")
# Categorize top companies by rough industry
industries = {
    'Tech': ['Oracle', 'Microsoft', 'Amazon', 'Google', 'Meta', 'Apple', 'NVIDIA', 'Adobe', 'Salesforce'],
    'Consulting': ['Deloitte', 'Accenture', 'Boston Consulting Group', 'McKinsey', 'PricewaterhouseCoopers'],
    'Aerospace': ['Boeing', 'Lockheed Martin', 'RTX', 'Northrop Grumman'],
    'Healthcare': ['CVS Health', 'Molina Healthcare', 'Intermountain Health', 'Highmark Health'],
    'Finance': ['JPMorgan Chase', 'Bank of America', 'Goldman Sachs', 'Morgan Stanley']
}

for industry, companies in industries.items():
    count = df[df['company_clean'].isin(companies)].shape[0]
    if count > 0:
        pct = (count / len(df)) * 100
        print(f"   {industry}: {count:,} jobs ({pct:.1f}%)")

print("\n" + "="*70)
print(" COMPANY NAME STANDARDIZATION COMPLETE")
print("="*70)


In [ ]:
print("\n Sample of standardized data:\n")
sample_cols = ['company', 'company_clean', 'role', 'location', 'has_ai_keywords', 'source']
df[sample_cols].head(15)
